# Quantum Computing - Simulation of Qubits

In [19]:
from matplotlib import pyplot as plt
import numpy as np
from functools import reduce
from math import log
from qiskit import *
from qiskit.visualization import plot_bloch_vector, plot_bloch_multivector
import itertools

## Conventions
A zero-qubit vector $|0⟩$ is with x and y components of the form $\begin{pmatrix} 1\\0 \end{pmatrix}$

A one-qubit vector $|1⟩$ is with x and y components of the form $\begin{pmatrix} 0\\1 \end{pmatrix}$

## Two Different Representations of Qubits
### A
A +qubit vector $|+⟩$ is with x and y components of the form $\begin{pmatrix} \frac{1}{\sqrt{2}}\\\frac{1}{\sqrt{2}} \end{pmatrix}$

A -qubit vector $|-⟩$ is with x and y components of the form $\begin{pmatrix} \frac{1}{\sqrt{2}}\\-\frac{1}{\sqrt{2}} \end{pmatrix}$

### B
A clockwisearrow_qubit vector $|↻⟩$ is with x and y components of the form $\begin{pmatrix} \frac{1}{\sqrt{2}}\\\frac{i}{\sqrt{2}} \end{pmatrix}$

A counterclockwisearrow_qubit vector $|↺⟩$ is with x and y components of the form $\begin{pmatrix} \frac{1}{\sqrt{2}}\\-\frac{i}{\sqrt{2}} \end{pmatrix}$

In [8]:
#Defining the various qubits based on different basis sets
zero_qubit=np.matrix('1; 0')
one_qubit=np.matrix('0; 1')
plus_qubit=1/np.sqrt(2)*np.matrix('1; 1')
minus_qubit=1/np.sqrt(2)*np.matrix('1; -1')
clockwisearrow_qubit=1/np.sqrt(2)*np.matrix([[1],[np.complex(0,1)]])    
counterclockwisearrow_qubit=1/np.sqrt(2)*np.matrix([[1],[-np.complex(0,1)]])

# Arbitrary State

A quantum state is usually just expressed as
$$
\alpha|0\rangle+\beta|1\rangle,
$$

Let us see how the state looks like with arbitrary $\alpha$ and $\beta$, when $\alpha = \frac{3}{5}e^{i\pi/7}$ and $\beta = -\frac{4i}{5}$. 

The more general form, a geoemtric representation of a quantum state is,
$$
e^{i\lambda}(\cos\frac{\theta}{2}|0\rangle+\sin\frac{\theta}{2}e^{i\phi}|1\rangle)
$$

So, we can map the values of $\alpha$ and $\beta$ into $\theta$, $\phi$ and $\lambda$ as,
$$\lambda=\pi/7$$ and 
$$\cos\frac{\theta}{2}=\frac{3}{5}$$. 

We want $\sin\frac{\theta}{2}$ to be positive, so $$\sin\frac{\theta}{2}=\frac{4}{5}$$. Hence,
$$
\theta=2\cdot sin^{-1}{\frac{4}{5}}.
$$

To evaluate $\phi$, we have
$$
e^{i\lambda}e^{i\phi}=-i \\
\therefore e^{i\phi} = -i/e^{i\pi/7}=-ie^{-i\pi/7}=e^{-i\pi/7-i\pi/2} \\
\therefore \phi=2\pi-\frac{\pi}{7}-\frac{\pi}{2}=\frac{19}{14}\pi,
$$

# Quantum State

In [33]:
#It takes an array of qubits and returns the state for all the qubits
def create_quantum_state(qubits):
    return reduce(lambda x,y:np.kron(x,y),qubits)

In [30]:
register_01=create_quantum_state([zero_qubit,one_qubit])

In [31]:
def get_nqubits_quantum_state(state):
    return int(log(state.size,2))


# Quantum Measurement 

In [34]:
def measure_in_01_basis(state):
    from random import random
    n_qubits=int(log(state.shape[0],2))
    probabilities=[(coeff*coeff.conjugate()).real for coeff in state.flat]
    rand=random()
    for idx,state_desc in enumerate([''.join(map(str,state_desc)) for state_desc in itertools.product([0, 1], repeat=n_qubits)]):
        if rand < sum(probabilities[0:(idx+1)]):
            return '|"%s">' % state_desc
    

In [35]:
print(measure_in_01_basis(create_quantum_state([one_qubit,zero_qubit,zero_qubit,one_qubit,one_qubit])))

|"10011">


In [36]:
for i in range(10):
    bell_state_phi_plus=(create_quantum_state([zero_qubit,zero_qubit])+ \
                         create_quantum_state([one_qubit,one_qubit]))/np.sqrt(2)
    print(measure_in_01_basis(bell_state_phi_plus))

|"00">
|"00">
|"00">
|"11">
|"00">
|"00">
|"00">
|"11">
|"00">
|"11">


# Separable States

In [37]:
def get_qubits_from_state(state):
    basis_states=[zero_qubit,one_qubit,plus_qubit,minus_qubit,clockwisearrow_qubit,counterclockwisearrow_qubit]
    for separated_state in itertools.product(basis_states, repeat=get_nqubits_quantum_state(state)):
        candidate_state=create_quantum_state(separated_state)
        if np.allclose(candidate_state,state):
            return separated_state


In [38]:
register_10011=create_quantum_state([one_qubit,zero_qubit,zero_qubit,one_qubit,one_qubit])
get_qubits_from_state(register_01)
get_qubits_from_state(register_10011)

(matrix([[0],
         [1]]), matrix([[1],
         [0]]), matrix([[1],
         [0]]), matrix([[0],
         [1]]), matrix([[0],
         [1]]))

In [39]:
non_separable_state_00_plus_11=1/np.sqrt(2)* \
(create_quantum_state([zero_qubit,zero_qubit])+ \
 create_quantum_state([one_qubit,one_qubit]))

In [40]:
ten_ninety_qubit=np.sqrt(0.1)*zero_qubit+np.sqrt(0.9)*one_qubit
create_quantum_state([ten_ninety_qubit,ten_ninety_qubit])

matrix([[0.1],
        [0.3],
        [0.3],
        [0.9]])

In [41]:
print(get_qubits_from_state(create_quantum_state([ten_ninety_qubit,ten_ninety_qubit])))

None


# Decoherence, $T_1$, and $T_2$

In [42]:
probability_state_one_after_point1millisecond=0.1
t=0.0001
# probability_state_one_after_point1millisecond = np.e**(-t/T1) so T1 = -t/np.log(probability_state_one_after_point1millisecond)
T1=-t/np.log(probability_state_one_after_point1millisecond)
print(T1)

4.342944819032519e-05
